In [ ]:
# Install and import necessary libraries
%pip install numpy pandas requests openpyxl

import pandas as pd
import requests
import string
import re

# API credentials
API_KEY = 'ada006c4a6c245fb12af2a0edd84d12d'
INST_TOKEN = 'daeb14a8b7d45dd0d76921ce1fd3e6a5'

# Load Excel data
faculty = pd.read_excel("ORCID_Faculty.xlsx")

# Clean and format faculty data
faculty['SCOPUS ID'] = faculty['SCOPUS ID'].astype(str).str.replace('.0', '', regex=False)
faculty['LAST NAME'] = faculty['LAST NAME'].apply(string.capwords)
faculty['FIRST, MIDDLE NAME(S)'] = faculty['FIRST, MIDDLE NAME(S)'].apply(string.capwords)
faculty['Name'] = faculty['FIRST, MIDDLE NAME(S)'] + ' ' + faculty['LAST NAME']

def get_abstract_and_keywords(api_key, scopus_id):
    """
    Fetch the abstract and author keywords for a given Scopus ID.
    """
    url = f'http://api.elsevier.com/content/abstract/scopus_id/{scopus_id}'
    headers = {
        'Accept': 'application/json',
        'X-ELS-APIKey': api_key,
        'X-ELS-Insttoken': INST_TOKEN
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        abstract = data.get("abstracts-retrieval-response", {}).get("item", {}).get("bibrecord", {}).get("head", {}).get("abstracts", None)

        keywords_data = data.get("abstracts-retrieval-response", {}).get("item", {}).get("bibrecord", {}).get("head", {}).get("citation-info", {}).get("author-keywords", {}).get("author-keyword", [])

        if isinstance(keywords_data, list):
            keywords = "; ".join([kw.get("$", "") for kw in keywords_data])
        elif isinstance(keywords_data, dict):
            keywords = keywords_data.get("$", "")
        else:
            keywords = ""

        return abstract, keywords
    else:
        return None, None

def get_publications(api_key, author_id, full_name, orcid_id):
    """
    Fetch all publications associated with a given author ID.
    Returns a DataFrame of publication details.
    """
    query = f'AU-ID%28{author_id}%29'
    url = f'https://api.elsevier.com/content/search/scopus?query={query}'
    headers = {
        'Accept': 'application/json',
        'X-ELS-APIKey': api_key,
        'X-ELS-Insttoken': INST_TOKEN
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        entries = data.get('search-results', {}).get('entry', [])
        if not entries:
            print("No publications found.")
            return []

        publications = []
        for pub in entries:
            title = pub.get('dc:title', 'N/A')
            affiliation = pub.get('affiliation', [{}])[0].get('affilname', 'N/A')
            journal = pub.get('prism:publicationName', 'N/A')
            doi = pub.get('prism:doi', 'N/A')
            year = pub.get('prism:coverDate', 'N/A')[:4]
            pub_type = pub.get('subtypeDescription', 'N/A')

            scopus_id = pub.get('dc:identifier', 'N/A').split(':')[1] if pub.get('dc:identifier') else None
            abstract, keywords = get_abstract_and_keywords(api_key, scopus_id) if scopus_id else ('N/A', 'N/A')

            # Avoid duplicate titles
            if title not in [p['Title'] for p in publications]:
                publications.append({
                    'Name': full_name,
                    'ORCID': orcid_id,
                    'Scopus ID': author_id,
                    'Title': title,
                    'Article Affiliation': affiliation,
                    'Journal': journal,
                    'Year': year,
                    'Type': pub_type,
                    'DOI': doi,
                    'Keywords': keywords,
                    'Abstract': abstract
                })

        return pd.DataFrame(publications)
    else:
        print(f"Error {response.status_code}: {response.text}")
        return []

def populate_data_frame(api_key, author_id, full_name, orcid_id):
    """
    Wrapper function to return publication data if author ID exists.
    """
    if author_id:
        return get_publications(api_key, author_id, full_name, orcid_id)
    return pd.DataFrame([])

def fix_punctuation(text):
    """
    Replaces encoded punctuation characters and strips HTML tags.
    """
    replacements = {
        "‚Äô": "’",
        "‚Äú": "“",
        "‚Äù": "”"
    }
    for old, new in replacements.items():
        text = text.replace(old, new)
    return re.sub(r"<sup>(\d+)</sup>", r"\1", text)

# Initialize final DataFrame
df = pd.DataFrame(columns=[
    'Name', 'ORCID', 'Scopus ID', 'Title', 'Article Affiliation',
    'Journal', 'Year', 'Type', 'DOI', 'Keywords', 'Abstract'
])

# Loop through faculty and collect publication data
for _, row in faculty.iterrows():
    publications = populate_data_frame(API_KEY, row["SCOPUS ID"], row["Name"], row["ORCID ID#"])
    df = pd.concat([df, publications], ignore_index=True)

# Cleaning: drop rows with missing Scopus ID
df = df.dropna(subset=['Scopus ID'])

# Remove unwanted punctuation in Abstracts
df['Abstract'] = df['Abstract'].apply(lambda x: re.sub(r"^[^\w\d]+", "", str(x)))

# Clean up Title formatting
df['Title'] = df['Title'].apply(fix_punctuation)

# Save to CSV
df.to_csv('scopus_publications_2_4.csv', index=False)
